# ¿Qué importancia tiene la fuente de ingresos del solicitante de un préstamo online?

## Objetivos

Al final de este caso, estará familiarizado con la regresión logística. En concreto, aprenderá a realizar e interpretar los coeficientes de un modelo de regresión logística para obtener información sobre su problema.

Durante el estudio del caso, se le presentará la regresión logística como medio para probar una hipótesis. A continuación, validará esta hipótesis convirtiendo la regresión logística en un clasificador y midiendo su rendimiento. La comprensión de la teoría y la interpretación de la regresión logística es una herramienta vital en el ámbito de la ciencia de datos, especialmente en la clasificación.

## Introducción

**Contexto empresarial.** Los préstamos en línea entre iguales (P2P) han facilitado la práctica de los préstamos. En esta forma de préstamo, no hay entrevista en persona y un prestatario puede simplemente rellenar un formulario en línea y obtener la aprobación de un préstamo. La información proporcionada únicamente por el prestatario es propensa a la exageración y la distorsión, especialmente cuando se trata de los ingresos. Todas las empresas de préstamos P2P se basan en un procedimiento bien diseñado que rechaza a los prestatarios con una alta probabilidad de no pagar sus deudas.

Rechazar a cualquiera que no tenga una fuente de ingresos verificada es una política relevante que las plataformas de préstamo pueden poner en marcha para ayudar a reducir la tasa de préstamos fallidos. Es natural sospechar que si no se puede verificar la fuente de ingresos de una persona, ésta podría no pagar el préstamo. Sin embargo, desde el punto de vista del prestatario, el proceso de verificación puede ser engorroso y llevar mucho tiempo, por lo que es posible que se cambie a otra plataforma debido a este inconveniente. 

**Problema empresarial.** Como científico de datos en una empresa emergente de préstamos P2P, debe responder a la siguiente pregunta: **"¿Debe la empresa verificar la fuente de ingresos de un solicitante de préstamo online antes de aprobar su préstamo? "**

**Contexto analítico.**  Los datos se han descargado de [LendingClub (LC) Statistics] (https://www.lendingclub.com/info/download-data.action) y contienen todos los préstamos emitidos entre 2007 y 2012 junto con su estado actual (totalmente pagados o cancelados). Hay ~50 variables que describen a los prestatarios y los préstamos; en aras de reducir la complejidad, la empresa ya ha realizado una preselección de estas variables basándose en los análisis existentes de LendingClub para seleccionar nueve variables relevantes, como los ingresos anuales, el grado de crédito de LendingClub, la propiedad de la vivienda, etc. Utilizaremos una nueva técnica, la **regresión logística**, para responder a nuestra pregunta.

El caso está estructurado de la siguiente manera:
 1. explorar los datos existentes para tener una idea aproximada de cómo interactúa cada variable del conjunto de datos con el estado actual del préstamo
 2. buscar los posibles efectos de confusión
 3. aprender los fundamentos de los modelos de regresión logística y, por último
 4. ajustar una serie de modelos de regresión logística para determinar si la verificación de la fuente de ingresos es o no significativa.

In [ ]:
# Load packages

import matplotlib.pyplot as plt
import numpy  as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as sfm

from matplotlib.widgets import Slider, Button, RadioButtons
from scipy import interp
from scipy.optimize import fsolve
from scipy.stats import chi2_contingency, ttest_ind
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from statsmodels.formula.api import ols


from IPython.display import display_html

## Exploración de datos

Antes de comenzar nuestro análisis exploratorio, echemos un vistazo a los datos que tenemos a nuestra disposición:

In [ ]:
#Load the data
df = pd.read_csv('Lending_club_cleaned_2.csv')

#Change loan_status, verification_status, emp_length, term and grade to category type 
df.loan_status = df.loan_status.astype(pd.api.types.CategoricalDtype(categories=['Charged Off', 'Fully Paid']))
df.verification_status = df.verification_status.astype(pd.api.types.CategoricalDtype(categories=['Not Verified', 'Source Verified', 'Verified']))
df.emp_length = df.emp_length.astype(pd.api.types.CategoricalDtype(categories=['< 1 year', '1 year', '2 years', '3 years', '4 years', \
                                                             '5 years', '6 years', '7 years', '8 years', '9 years', \
                                                             '10+ years']))
df.home_ownership = df.home_ownership.astype(pd.api.types.CategoricalDtype(categories=['RENT','MORTGAGE','OWN','OTHER']))
df.term = df.term.astype(pd.api.types.CategoricalDtype(categories=[' 36 months', ' 60 months']))
df.grade = df.grade.astype(pd.api.types.CategoricalDtype(categories=['A','B','C','D','E','F','G']))

#In addition, the original data in int_rate contains strings of the form 'x.xx%',
#we remove the % and change the vaules to float:
df.int_rate = df.int_rate.str.rstrip('%').astype('float')

In [ ]:
df.shape

In [ ]:
df.head(10)

Tenemos 38705 registros de transacciones pasadas en este conjunto de datos. Cada registro corresponde a un préstamo aprobado. La primera columna indica si el prestatario pagó el préstamo (totalmente pagado) o no (cargado). Las descripciones de las otras nueve columnas son las siguientes:

|      annual_inc     |                                                 The self-reported annual income provided by the borrower during registration.                                                |
|:-------------------:|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| verification_status |                                          Indicates if income was verified by LC, not verified, or if the income source was verified                                          |
|      emp_length     |                       Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.                      |
|    home_ownership   |             The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER            |
|       int_rate      |                                                                           Interest Rate on the loan                                                                          |
|      loan_amnt      | The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value. |
|       purpose       |                                                           A category provided by the borrower for the loan request.                                                          |
|         term        |                                             The number of payments on the loan. Values are in months and can be either 36 or 60.                                             |
|        grade        |                                                                            LC assigned loan grade                                                                            |

### Relaciones entre `loan_status` y otras variables

En primer lugar, realizamos un EDA para examinar la relación por pares entre "estado del préstamo" y cada una de las demás variables. En el conjunto de datos tenemos variables discretas y continuas. Empecemos a explorar las variables `anual_inc`, `int_rate` y `loan_amnt` en los usuarios que pagaron totalmente sus deudas frente a los que no lo hicieron:

In [ ]:
fig, ax=plt.subplots(1,3, figsize=(18,8))
sns.boxplot(y='annual_inc', x='loan_status', data=df, ax=ax[0]).set_yscale('log')
sns.boxplot(y='int_rate', x ='loan_status', data = df, ax=ax[1])
sns.boxplot(y='loan_amnt', x='loan_status', data=df, ax=ax[2])

### Ejercicio 1:

#### 1.1

¿Qué observas en los gráficos de caja anteriores? 

#### 1.2

Considere la afirmación:

> "A partir de estos gráficos podemos concluir que los ingresos anuales no desempeñan un papel significativo a la hora de decidir si un usuario va a devolver o no su préstamo."

¿Cree que esta afirmación es correcta? ¿Cómo intentarías comprobarlo? 

In [ ]:
a=df[df['loan_status']=="Charged Off"]['annual_inc']
b=df[df['loan_status']=="Fully Paid"]['annual_inc']
T,p=ttest_ind(a,b)
p

Podemos utilizar tablas de contingencia para examinar la relación entre dos variables discretas. Se puede realizar una prueba de chi-cuadrado basada en la tabla de contingencia para verificar si la relación observada es estadísticamente significativa. La siguiente función genera la tabla de contingencia entre `estado_de_préstamo` y otra variable dada llamada `var`:

In [ ]:
def get_ct(df, var):
    ct_res = pd.crosstab(df[var], df['loan_status'], margins=True)
    ct_res['Fully Paid(%)'] = round( ct_res['Fully Paid']/ct_res['All']*100, 2)
    return ct_res.drop(columns='All')

Por ejemplo, calculemos la tabla de contingencia de `estado_de_préstamo` frente a `estado_de_verificación`:

In [ ]:
get_ct(df, 'verification_status')

### Ejercicio 2:



#### 2.1

¿Qué se desprende del cuadro anterior? ¿Podemos concluir que la verificación de la fuente de ingresos no es necesaria para nuestro cliente?

**Answer.** One possible solution is shown below:

From the table, `verification_status` seems to have a counter-intuitive association with `loan_status` - the less reliable the income information is, the more likely a user would fully repay the debt, as reflected by the decreasing trend of fully paid proportions when `verification_status increases` trends from not verified to fully verified. At first glance, this indicates that source verification may be a waste of money for the client. However, we must be careful with our conclusions as this table is not taking into account other possible factors that may have an effect on users paying off their debts. For instance, it may be the case that a considerable portion of loans with smaller interest rates were given to non-verified users, which made it easier to pay off the loan.

#### 2.2

Utilice una prueba de chi-cuadrado para verificar si las asociaciones observadas entre "estado de verificación" y "estado de préstamo" son estadísticamente significativas.

**Sugerencia:** Utilice `chi2, p, dof, ex=chi2_contingency(pd.crosstab(index=x,columns=y))` para realizar una prueba de chi-cuadrado para un par de datos dado `x, y`.

**Una posible solución se muestra a continuación:

In [ ]:
chi2, p, dof, ex = chi2_contingency(pd.crosstab(index=df['verification_status'], columns=df['loan_status']))
print('verification_status: p-value of chisquare test =', p)

La prueba de chi-cuadrado verifica que las asociaciones entre "estado de verificación" y "estado de préstamo" son realmente significativas desde el punto de vista estadístico.

#### 2.3

Considere las siguientes afirmaciones:

> I. "Dado que la prueba de chi-cuadrado para la tabla de contingencia de `estado_de_verificación` frente a `estado_de_préstamo` es significativa, la tendencia decreciente de la probabilidad de pago no es probable que se deba al azar".

> II. "Dado que existe una diferencia significativa entre la probabilidad de pago total en los usuarios sin ingresos verificados y los usuarios con ingresos verificados según la prueba de chi-cuadrado, deberíamos aplicar siempre la verificación de ingresos."

¿Son correctas las afirmaciones?

**Answer.** One possible solution is shown below:

Statement I is not correct. The chi-sqaure test is not a trend test, it only suggests that there is significant difference in the probability of pay-off among the three levels of `verification_status`, but it cannot further verify that the differences form a specific trend.

Statement II is not correct since it implies a causal relationship between `verification_status` and `loan_status`. There is not enough evidence to deduce this, and we should first assess if there are potential confounding variables like we did in previous cases before making such a strong conclusion.

### Tablas de contigencia estratificadas de `estado_de_verificación` frente a `estado_de_préstamo`

Como vimos en el ejercicio anterior, tenemos que ajustar por otras variables para eliminar posibles impactos de confusión. Si queremos saber si el `estado de la verificación` está realmente asociado con el `estado del préstamo` como indica la tabla de contigencia anterior, debemos considerar la estratificación de la tabla de contingencia entre el `estado del préstamo` y el `estado de la verificación` por otras variables que también están asociadas con el `estado del préstamo`.

### Ejercicio 3:

Escriba una función que genere una tabla de contingencia para `estado de verificación` frente a `estado de préstamo` estratificada por la `stra_var`, el argumento de la función. Asegúrese de incluir una columna para la proporción de usuarios totalmente pagados en cada nivel de "estado de verificación". Utilice esta función para examinar la relación entre el "estado de verificación" y el "estado de los préstamos" cuando se ajuste por la "propiedad de la vivienda" y el "plazo". ¿Seguimos viendo la misma tendencia a través de los diferentes niveles de "propiedad de la vivienda" y "plazo"?

**Sugerencia:** Cuando se utiliza el parámetro `index` de `pd.crosstab()` con dos variables, la tabla de contingencia se estratifica por la primera variable en `index`.

**Una posible solución se muestra a continuación:

In [ ]:
def get_ct_stra(stra_var):
    ct_stra = pd.crosstab(index=[stra_var,df.verification_status], columns = df.loan_status, margins = True)
    ct_stra['Fully Paid (%)'] = round(ct_stra['Fully Paid']/ct_stra['All']*100,2)
    return ct_stra.drop(columns='All').drop('All', level=0)

In [ ]:
get_ct_stra(df.home_ownership)

In [ ]:
get_ct_stra(df.term)

Podemos ver que la tendencia se mantiene cuando se estratifica por "propiedad del hogar". La proporción de usuarios de pago completo disminuye a medida que el estado de verificación es menos fiable, independientemente de los niveles de "propiedad de la vivienda". La excepción es el nivel "OTRO". Pero como tenemos muy pocos usuarios allí, no deberíamos confiar demasiado en los resultados. La tendencia ya no se mantiene en ambos niveles de "plazo". Podemos ver que la proporción de usuarios totalmente pagados con un plazo de préstamo más corto (36 meses) sigue siendo la misma en los distintos niveles de "estado de verificación", en torno al 88%.

### Ejercicio 4:

Comparando las tablas de contingencia estratificadas con las homólogas no estratificadas, ¿cuál de las siguientes conclusiones es correcta? Seleccione todas las que correspondan.

I. Basándose en la tabla de contigencia estratificada, la diferencia en la probabilidad de pago completo entre los usuarios con y sin verificación de ingresos podría ser atribuible al efecto del plazo del préstamo en la probabilidad de pago completo.

II. Basándose en la tabla de contigencia estratificada, la diferencia en la probabilidad de pago completo entre los usuarios con y sin verificación de ingresos no es probablemente atribuible al efecto de la propiedad de la vivienda en la probabilidad de pago completo.

III. Supongamos que los usuarios sin ingresos verificados son más propensos a pedir préstamos a corto plazo en comparación con los usuarios con ingresos verificados y que los usuarios con préstamos a corto plazo son más propensos a pagar sus deudas. Si, de hecho, el estado de verificación de los ingresos no tiene ningún efecto sobre la probabilidad de devolver la deuda, los usuarios sin ingresos verificados siguen teniendo, al margen, más probabilidades de pagar su deuda.

## Extracción de información adicional con regresión lineal

A partir de la prueba de chi-cuadrado, sabemos que el estado de verificación está marginalmente asociado a la probabilidad de pago. Observamos en la tabla de contingencia que esta probabilidad aumenta cuando la información sobre los ingresos es menos fiable. Sin embargo, esta observación no puede verificarse sólo con la prueba estándar de chi-cuadrado.
    
Como primera aproximación, vamos a crear un modelo lineal que pretende explicar `estado_de_préstamo` utilizando las variables dadas en nuestro conjunto de datos. Tenga en cuenta que "estado_de_préstamo" es una variable categórica, por lo que es necesario dummificarla primero. Además, dado que las variables "estado_de_verificación", "grado" y "término" son ordinales, podríamos utilizar también sus códigos categóricos (esto supondría que los saltos en las categorías son iguales, lo que podría ser razonable para algunas de estas variables). Por último, para evitar que aparezcan números grandes en los coeficientes de la regresión, vamos a normalizar los datos numéricos. Este procedimiento no afecta al resultado de la regresión - OLS es invariante bajo transformaciones lineales:

In [ ]:
df2=df.copy()

var_names=['loan_status','verification_status','grade','term']
for var in var_names:
    df2[var]=df2[var].cat.codes
    
var_names2=['annual_inc','int_rate','loan_amnt']
for var in var_names2:
    df2[var]=(df2[var]-df2[var].mean())/df2[var].std()

Consideremos un modelo sencillo con las variables `annual_inc`, `verification_status`, `loan_amnt`, `int_rate` y `term`:

In [ ]:
model1 = 'loan_status~verification_status+annual_inc+loan_amnt+int_rate+term'
lm1   = sfm.ols(formula = model1, data = df2).fit()
print(lm1.summary())

### Ejercicio 5:



#### 5.1

¿Qué puedes concluir de estos resultados?

#### 5.2

¿Qué desventajas crees que tiene un regresor lineal en esta situación?

**Posible Respuesta.**

Utilizar un modelo de regresión lineal en este contexto puede ser un poco desventajoso ya que nuestra variable objetivo es binaria. Esta restricción adicional nunca se utiliza cuando se construyen modelos lineales. Deberíamos ser capaces de explicar mejor el comportamiento de `loan_status` con modelos que tengan en cuenta esta información vital.

In [ ]:
p=np.random.rand(100)
x=np.random.rand(100)
y=np.logical_or(np.logical_and(x<=0.5, p>0.8), np.logical_and(x>0.5, p<=0.8)).astype(int)

sns.regplot(x,y)

## Regresión logística: ampliación del modelo lineal a resultados binarios

El modelo de regresión logística es una versión modificada del modelo de regresión lineal estándar que se ocupa de los problemas de clasificación, en los que los resultados son variables binarias (en este caso, si el préstamo se pagó o se canceló). Para entender un modelo de regresión logística, introduzcamos primero la idea de **razón de oportunidad** (Odds en inglés). Las odds de un suceso son la relación entre su probabilidad de éxito y su probabilidad de fracaso. En nuestro caso, las probabilidades de pagar completamente la deuda se definen como

$$
\text{Odds}(\text{Pago total}) = \frac{p}{1-p}.
$$

donde $p$ es la probabilidad de pago.

Resulta que es más fácil trabajar con odds que con las probabilidades. Para investigar nuestra pregunta sobre la tendencia de la probabilidad de pago $p$, suponemos que las probabilidades son una función del estado de verificación de los ingresos:

\begin{equation}
\log(\text{Odds}) = \beta_0 + \beta_1*\text{verification_status}. \tag{1}
\end{equation}

Para reflejar que existe una tendencia sobre los diferentes estados de verificación, codificamos diferentes niveles de `estado_de_verificación` utilizando números enteros: "No verificado" como 0, "Fuente verificada" como 1 y "Verificado" como 2; es decir, estamos renombrando las categorías de una variable categórica para reflejar una clara tendencia numérica. Afirmamos que este esquema de codificación implica que si $\beta_1$ es positivo, entonces hay una tendencia creciente de la probabilidad de pago a medida que el estado de verificación de los ingresos es más fiable. Por otro lado, si $\beta_1$ es negativo, ocurre lo contrario.

Una vez que se ajusta el modelo en (1), podemos realizar una prueba para examinar si $\beta_1$ es significativamente diferente de cero. Obsérvese que si $\beta_1=0$, entonces $p$ es el mismo para diferentes niveles de `estado de verificación` y, por tanto, no hay tendencia. Si $\beta_1$ es significativamente diferente de cero, podemos utilizar el signo estimado de $\beta_1$ para determinar la dirección de la tendencia.

Veamos cómo se puede ejecutar un modelo de regresión logística utilizando la función `statsmodels.api` `Logit`:

In [ ]:
# code the discrete variable by the specification above
df_log1 = pd.DataFrame(columns=['verification_status','loan_status'])
df_log1['verification_status'] = df.verification_status.cat.codes
df_log1['loan_status'] = df.loan_status.cat.codes
df_log1['Intercept'] = 1

logit = sm.Logit(df_log1['loan_status'], df_log1[['Intercept','verification_status']])
logit_res = logit.fit()
logit_res.summary()

### Interpretación de los coeficientes en la salida de una regresión logística

Aunque el modelo de regresión logística es más complejo que la regresión lineal estándar, el signo del coeficiente de regresión sigue representando la dirección de la influencia de una variable específica: un coeficiente positivo significa que la probabilidad de pago aumentará si la variable asociada aumenta y viceversa.

La otra parte importante de la información al interpretar un modelo de regresión logística es el valor $p$ de cada coeficiente de regresión. El valor p$ indica el resultado de la siguiente prueba de hipótesis:

$$
H_0: \beta = 0~vs.~ H_A: \beta \neq 0.
$$

La prueba determina si la diferencia en la probabilidad de pago está asociada a los cambios en la variable correspondiente.

A veces puede encontrar que tanto el valor $p$ como el tamaño del efecto estimado (coeficiente de regresión) para una variable son pequeños. Debe tener cuidado con esta situación, ya que los valores $p$ no son la única forma de determinar si las variables son importantes.

Por ejemplo, a partir de los resultados mostrados anteriormente para el modelo `logit_res`, los coeficientes indican que cuando el `estado de verificación` aumenta en 1, las probabilidades de pago disminuyen en un 14%. Dado que el valor $p$ del coeficiente es inferior a 0,05, podemos concluir que la tendencia decreciente que observamos en la tabla no se debe probablemente al azar.

Un resultado importante es el `Pseudo R-squ.`. Esta métrica es similar a la R-cuadrado para los modelos lineales. Si este número es grande, entonces las variables del modelo explican una gran parte de los factores que impulsan la tendencia de la gente a pagar su deuda. En nuestro caso, la métrica es sólo del 0,2%, lo que significa que tenemos mucho margen de mejora.

### Interpretación geométrica de la regresión logística

La regresión logística se convierte fácilmente en un **clasificador** eligiendo un **límite de decisión**. Es decir, escogiendo un valor de corte $c=\log\left(\frac{p}{1-p}\right)$, podemos clasificar usando la siguiente regla:

> si $\beta_0 + \beta_1*\text{verification_status} > c$, entonces se trata de un pago completo; o bien

> si $\beta_0 + \beta_1*\text{verification_status} \leq c$ entonces Charged off

La curva que separa a los morosos de los no morosos, definida por la ecuación $\beta_0 + \beta_1$ `estado_de_verificación` $=\log\left(\frac{p}{1-p}\right)$, es la frontera de decisión determinada por el modelo (1) y el valor de corte $p$. Por ejemplo, utilizando los resultados anteriores, si fijamos $p = 0,5$, entonces  

> El usuario se clasificaría como Totalmente pagado si $\text{verification_status} >\frac{-\beta_0}{\beta_1}\approx 13.334931506849317$

> El usuario se clasificaría como cargado si  $\text{verification_status} \leq \frac{-\beta_0}{\beta_1}\approx 13.334931506849317$

Evidentemente, esta frontera de decisión es inútil, ya que sabemos que los valores de `estado_de_verificación` son siempre como máximo $2$ para cada usuario. Por lo tanto, tendríamos que averiguar qué valor de $p$ es el que nos da la mejor clasificación posible. Ampliaremos esta cuestión más adelante.  

### Exercise 6:

#### 6.1

Ejecute un modelo de regresión logística para `estado_de_préstamo` utilizando como regresores las variables `amnt_de_préstamo` y `tasa_de_intensidad`. Normalice estas dos variables antes de ajustar el modelo.

#### 6.2

Utilice los coeficientes encontrados en el Ejercicio 6.1 para ejecutar el código siguiente. Mueva el cursor para ver los cambios en la frontera de decisión.

In [ ]:
b_Intercept=1 #Here your coeff
b_loan_amnt=1 #Here your coeff
b_int_rate=1 #Here your coeff


df6=df.copy()
df6['loan_status'] = df6.loan_status.cat.codes
df6['loan_amnt']=(df6['loan_amnt']-df6['loan_amnt'].mean())/df6['loan_amnt'].std()
df6['int_rate']=(df6['int_rate']-df6['int_rate'].mean())/df6['int_rate'].std()
df6['Intercept'] = 1

import ipywidgets as widgets
from ipywidgets import interact
%matplotlib notebook

fig, ax = plt.subplots()
sns.scatterplot(x='int_rate',y='loan_amnt',hue='loan_status',data=df6,alpha=0.1,ax=ax)
plt.xlim(-2,3)
plt.ylim(-2,3)
plt.legend()


p=0.5
x1=np.arange(-2,40,3)
y1=(1/b_loan_amnt)*(np.log(p/(1-p))-b_Intercept-b_int_rate*x1)
l, = plt.plot(x1, y1, lw=2, color='k', label='Current Decision boundary')

def update(p=0.5):
    y1=(1/b_loan_amnt)*(np.log(p/(1-p))-b_Intercept-b_int_rate*x1)
    l.set_ydata(y1)
    fig.canvas.draw_idle()

interact(update, p=widgets.FloatSlider(value=p,min=0.01,max=0.99,step=0.01))

In [ ]:
%matplotlib inline

### Evaluación del modelo de regresión logística

Un inconveniente importante de la pseudo métrica $R^2$ es que no puede traducirse directamente en la precisión de la predicción de un modelo. Recordemos que el objetivo de todo nuestro análisis es predecir si un usuario va a pagar o no su deuda. Pero, ¿cómo evaluamos si la predicción es precisa o no? Esta pregunta no es tan sencilla como parece y la razón es que los modelos logísticos sólo producen probabilidades mientras que lo que observamos son etiquetas binarias.

Una solución directa es que se puede seleccionar un valor de corte para la probabilidad predicha, de forma que las personas con probabilidades inferiores al valor de corte se consideren morosos. Esto convierte las probabilidades en etiquetas binarias, tras lo cual podemos ver si las etiquetas predichas son las mismas que las observadas para medir la precisión de la predicción.

### Pregunta:

¿Cómo podría seleccionar el mejor valor de corte para un determinado modelo de regresión logística?

### Ejercicio 7:

Calcule el TPR (True Positive Rate) y el TNR (True Negative Rate) para el modelo del Ejercicio 6 utilizando valores de corte $p = 0,8, 0,85, 0,9$. ¿Cuál de estos valores de corte le parece mejor? ¿Por qué?

In [ ]:
b_Intercept=1.9277 
b_loan_amnt=0.0399 
b_int_rate=-0.5862 

def rates(p):
    a= df6.loan_status
    b=(b_Intercept+b_loan_amnt*df6['loan_amnt']+b_int_rate*df6['int_rate']>np.log(p/(1-p))).astype('int')
    
    TPR=(a & b).sum(axis=0)/a.sum(axis=0)
    TNR=(1-a & 1-b).sum(axis=0)/(1-a).sum(axis=0)
    return [TPR,TNR]

print('p=0.8 ',rates(0.8))
print('p=0.85 ',rates(0.85))
print('p=0.9 ', rates(0.9))

A partir de aquí, vemos que la elección de un punto de corte de $p = 0,8$ da un TPR alto y un TNR bajo. Esto significa que este punto de corte clasificaría correctamente a la mayoría de los buenos prestatarios, pero también pasaría por alto un número considerable de morosos. Como ya se ha dicho, esto puede ser problemático, ya que podríamos acabar perdiendo dinero. 

En el otro extremo, $p = 0,9$ detectaría correctamente una buena parte de los morosos, pero también denegaría muchos préstamos a usuarios que probablemente pagarán. Así, podríamos acabar perdiendo clientes a favor de nuestros competidores y perder la oportunidad de trabajar con buenos prestatarios.

En un enfoque más conservador, $p = 0,85$ detectaría correctamente el 66% de los buenos prestatarios y alrededor del 54% de los morosos. Esta podría ser una alternativa, ya que no perderíamos gran parte de nuestros clientes potenciales y nuestro riesgo estaría relativamente controlado. No obstante, probablemente deberíamos consultar esta decisión con nuestro cliente, ya que podría preferir un enfoque más seguro o más arriesgado.

### La curva ROC (Receiver Operating Characteristic)

Hemos visto el gran potencial de considerar el TPR y el TNR para ayudar a seleccionar el umbral adecuado para nuestros modelos. Sin embargo, no tiene mucho sentido calcularlos para valores de corte únicos. Deberíamos resumir la precisión de la predicción en todos los valores de corte diferentes para producir una evaluación unificada de nuestro modelo. Este es el motivo de la **curva (ROC)**.

La curva ROC traza 1 - TNR contra TPR e ilustra el comportamiento de un modelo logístico a medida que cambia el punto de corte. Vamos a trazar la curva ROC para el modelo del Ejercicio 6:

In [ ]:
#This function calculates the TPR and TNR for given p
def rates(p):
    a= df6.loan_status
    b=(b_Intercept+b_loan_amnt*df6['loan_amnt']+b_int_rate*df6['int_rate']>np.log(p/(1-p))).astype('int')
    
    TPR=(a & b).sum(axis=0)/a.sum(axis=0)
    TNR=(1-a & 1-b).sum(axis=0)/(1-a).sum(axis=0)
    return [TPR,TNR]

#Then, we use this function to plot the ROC curve
x=[]
y=[]
for p in np.arange(0.01,0.99,0.01):
    z=rates(p)
    y.append(z[0])
    x.append(1-z[1])
    
fig, ax = plt.subplots()

ax.plot(x,y, label=f'ROC curve for model in Ex6')
ax.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--', label='Random guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Classification of Defaulters')
plt.legend(loc='lower right')

### Ejercicio 8:

Dibuje las curvas ROC para `logit_res` y el modelo del Ejercicio 6 en el mismo gráfico. ¿Puede determinar qué modelo es mejor observando únicamente estos gráficos? Proponga una forma de evaluar diferentes modelos utilizando las curvas ROC.

**Sugerencia:** Utilice `logit_res.predict(df_log1[['Intercept','verification_status']])` para calcular las probabilidades predichas del modelo `logit_res`. Llame a este resultado `predict_p` y utilice la función `roc_cure(df_log1['estado_de_préstamo'],predict_p)` para calcular las coordenadas de la curva ROC del modelo.

### Comparing models using the area under the curve (AUC)

A grandes rasgos, el AUC indica la probabilidad de que un moroso seleccionado al azar tenga un 𝑝 menor que un no moroso seleccionado al azar.

Si el AUC se aproxima a uno, la regla se aproxima a una regla perfecta. La curva ROC para una regla de adivinación aleatoria (establecer 𝑝=0,5 para todos los solicitantes) es la línea diagonal en el gráfico y suele servir como línea de base para comprobar si el modelo de clasificación aprende alguna información de los datos por encima de la pura aleatoriedad. Podemos calcular el AUC después del cálculo de la curva ROC utilizando la función auc de sklearn.metrics:

In [ ]:
#Note, here we assumed you named the roc curves of Exercise 8 roc_p and roc_2_p

auc_p = auc( roc_p[0], roc_p[1] )
auc_2_p = auc( roc_2_p[0], roc_2_p[1] )

print('AUC of `log_res`: ', auc_p)
print('AUC of the model of Ex 6: ', auc_2_p)

## Añadiendo variables adicionales al modelo

En las secciones anteriores, hemos observado una tendencia decreciente en la probabilidad de pago asociada a una información de ingresos más fiable. Sin embargo, esta tendencia es contraria a la intuición. Esto podría deberse a que muchos otros factores también están asociados a la probabilidad de pago, como hemos visto en los análisis exploratorios y en cierto modo en nuestro modelo de regresión lineal ingenuo. Si no se tienen en cuenta estos factores, las conclusiones sobre el efecto del "estado de verificación" serán completamente diferentes.

Utilizando la función `get_ct`, que se utiliza para calcular las tablas de contingencia de las variables categóricas, no es difícil ver que, excepto `annual_inc`, todas las demás variables del conjunto de datos están correlacionadas con `verification_status`. Por ejemplo, parece haber una fuerte correlación entre el "estado de verificación" y el "préstamo". Las personas sin fuente de ingresos verificada tienden a pedir menos dinero prestado que las personas con fuente de ingresos verificada. Esta podría ser la razón de la tendencia a la baja asociada al `estado de verificación`, ya que es más probable que se paguen los préstamos más pequeños. Incorporemos ahora todas estas variables (incluyendo por ahora "annual_inc") en el modelo logístico junto con "verification_status" y veamos si la tendencia decreciente asociada a "verification_status" sigue presente:

In [21]:
# Preprocessing the variables
df_log2 = pd.concat([(df.loan_amnt - df.loan_amnt.mean())/df.loan_amnt.std(),\
                     (df.int_rate - df.int_rate.mean())/df.int_rate.std(),\
                     (df.annual_inc - df.annual_inc.mean())/df.annual_inc.std(),\
                     pd.get_dummies(df.home_ownership, prefix='home', drop_first=True), \
                     pd.get_dummies(df.purpose, prefix='purpose', drop_first=True), \
                     pd.get_dummies(df.grade, prefix='grade',drop_first=True)], axis=1)
df_log2['verification_status'] = df.verification_status.cat.codes
df_log2['emp_length'] = df.emp_length.cat.codes
df_log2['term'] = df.term.cat.codes
df_log2['Intercept'] = 1

In [22]:
logit_full1 = sm.Logit(df.loan_status.cat.codes, df_log2)
logit_full1_res = logit_full1.fit()
logit_full1_res.summary()

Optimization terminated successfully.
         Current function value: 0.378075
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                38705
Model:                          Logit   Df Residuals:                    38676
Method:                           MLE   Df Model:                           28
Date:                Tue, 20 Sep 2022   Pseudo R-squ.:                 0.06869
Time:                        13:51:49   Log-Likelihood:                -14633.
converged:                       True   LL-Null:                       -15713.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
loan_amnt                     -0.0075      0.019     -0.389      0.697      -0.045       0.030
int_rate                      -0.4050      0.054     -7.454      0.000      -0.511      -0.299
annual_inc                     0.3615      0.031     11.641      0.000       0.301       0.422
home_MORTGAGE                  0.0382      0.035      1.095      0.273      -0.030       0.107
home_OWN                       0.0209      0.060      0.348      0.728      -0.097       0.138
home_OTHER                    -0.4454      0.268     -1.660      0.097      -0.971       0.080
purpose_credit_card            0.0364      0.100      0.363      0.717      -0.160       0.233
purpose_debt_consolidation    -0.2190      0.091     -2.405      0.016      -0.398      -0.041
purpose_educational           -0.5663      0.178     -3.186      0.001      -0.915      -0.218
purpose_home_improvement      -0.1624      0.107     -1.523      0.128      -0.371       0.047
purpose_house                 -0.3556      0.171     -2.077      0.038      -0.691      -0.020
purpose_major_purchase        -0.0024      0.114     -0.021      0.983      -0.227       0.222
purpose_medical               -0.3945      0.141     -2.794      0.005      -0.671      -0.118
purpose_moving                -0.4147      0.150     -2.766      0.006      -0.708      -0.121
purpose_other                 -0.4050      0.099     -4.105      0.000      -0.598      -0.212
purpose_renewable_energy      -0.7242      0.286     -2.530      0.011      -1.285      -0.163
purpose_small_business        -0.9414      0.105     -8.961      0.000      -1.147      -0.735
purpose_vacation              -0.4371      0.179     -2.439      0.015      -0.788      -0.086
purpose_wedding                0.1348      0.142      0.948      0.343      -0.144       0.414
grade_B                       -0.2733      0.074     -3.709      0.000      -0.418      -0.129
grade_C                       -0.3549      0.103     -3.448      0.001      -0.557      -0.153
grade_D                       -0.3775      0.132     -2.860      0.004      -0.636      -0.119
grade_E                       -0.2973      0.160     -1.859      0.063      -0.611       0.016
grade_F                       -0.3238      0.194     -1.669      0.095      -0.704       0.057
grade_G                       -0.1802      0.238     -0.757      0.449      -0.647       0.286
verification_status            0.0082      0.020      0.411      0.681      -0.031       0.047
emp_length                    -0.0147      0.004     -3.301      0.001      -0.023      -0.006
term                          -0.4520      0.037    -12.081      0.000      -0.525      -0.379
Intercept                      2.6163      0.123     21.216      0.000       2.375       2.858
==============================================================================================
"""

### Ejercicio 9:

Según los resultados del modelo logístico con todas las variables incluidas, ¿cuál de las siguientes conclusiones es correcta? Seleccione todas las que correspondan.

I. No hay pruebas que apoyen la asociación entre la probabilidad de pago y el estado de verificación después de tener en cuenta otras variables.

II. Los préstamos para pequeñas empresas tienen la menor probabilidad de reembolso.

III. La probabilidad de amortización de los usuarios con grado B de LendingClub disminuye en un 24% en comparación con los usuarios con grado A después de tener en cuenta otras variables.

### ¿Por qué observamos una tendencia marginal asociada a verification_status?

Curiosamente, encontramos que el coeficiente de regresión para `estado de verificación` es ahora positivo y el valor $p$ para él es muy grande (0,68). Esto indica que, tras ajustar todas las demás variables del conjunto de datos, el "estado de verificación" ya no está significativamente asociado a la probabilidad de pago. Averigüemos ahora qué variables de confusión fueron las responsables de introducir la tendencia decreciente inicial que observamos.

Podemos investigar este problema utilizando los resultados de la regresión junto con la correlación entre "estado de verificación" y todas las demás variables. En primer lugar, veamos la relación por pares entre "estado de verificación" y todas las demás variables:

In [23]:
def get_rctable(var1, var2):
    res = pd.crosstab(df[var2], df[var1])
    chi2, p, dof, ex = chi2_contingency(res)
    output = round( res.div(res.sum(axis = 1),axis = 0)*100, 2 )
    return output.style.set_caption(f'{var1} vs. {var2}: Chi-square p-value={p:.2f}')

In [24]:
display(get_rctable('emp_length', 'verification_status'))
display(get_rctable('home_ownership', 'verification_status'))
display(get_rctable('purpose', 'verification_status'))
display(get_rctable('term', 'verification_status'))
display(get_rctable('grade', 'verification_status'))

emp_length,< 1 year,1 year,2 years,3 years,4 years,5 years,6 years,7 years,8 years,9 years,10+ years
verification_status,,,,,,,,,,,
Not Verified,12.090000,9.410000,12.430000,11.100000,8.980000,8.590000,5.670000,4.440000,3.820000,3.330000,20.130000
Source Verified,15.080000,8.520000,11.220000,10.710000,8.720000,8.480000,5.770000,4.650000,3.540000,3.030000,20.250000
Verified,8.990000,6.920000,10.020000,9.810000,8.940000,8.350000,5.880000,4.720000,4.080000,3.320000,28.980000


home_ownership,RENT,MORTGAGE,OWN,OTHER
verification_status,,,,
Not Verified,49.730000,41.590000,8.360000,0.320000
Source Verified,52.420000,40.020000,7.450000,0.100000
Verified,41.620000,52.180000,5.920000,0.290000


purpose,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding
verification_status,,,,,,,,,,,,,,
Not Verified,4.770000,13.470000,43.510000,1.350000,7.170000,0.830000,6.150000,1.800000,1.420000,12.100000,0.190000,3.950000,0.990000,2.290000
Source Verified,4.590000,12.030000,46.270000,0.330000,7.350000,1.000000,6.020000,2.080000,2.000000,9.310000,0.380000,4.570000,1.100000,2.980000
Verified,2.110000,12.950000,52.900000,0.510000,7.930000,1.080000,4.130000,1.360000,1.030000,7.480000,0.210000,5.530000,0.650000,2.130000


term,36 months,60 months
verification_status,,
Not Verified,85.790000,14.210000
Source Verified,69.460000,30.540000
Verified,59.010000,40.990000


grade,A,B,C,D,E,F,G
verification_status,,,,,,,
Not Verified,30.340000,31.980000,20.830000,11.630000,3.930000,1.030000,0.270000
Source Verified,25.300000,29.430000,19.950000,13.970000,7.760000,2.870000,0.710000
Verified,17.800000,28.500000,20.530000,15.520000,11.310000,4.710000,1.640000


In [ ]:
sns.boxplot(y='loan_amnt', x='verification_status', data = df)

In [ ]:
sns.boxplot(y='annual_inc', x='verification_status', data = df).set_yscale('log')

In [ ]:
sns.boxplot(y='int_rate', x='verification_status', data = df)

### Ejercicio 10:

Según las relaciones que hemos observado en las tablas de contingencia y los gráficos de caja anteriores, ¿cuál de las siguientes afirmaciones es correcta? Seleccione todas las que correspondan.

A. La tendencia marginal es que las personas con ingresos verificados tienden a pedir préstamos con tipos de interés más altos

B. La tendencia marginal es que las personas con ingresos verificados tienden a tener una mayor duración del empleo

C. La tendencia marginal es que las personas con ingresos verificados tienden a tener hipotecas

## ¿Necesitamos incluir `verification_status`?

De lo anterior, podríamos llegar a la conclusión de que `verification_status` no importa cuando queremos predecir si un prestatario va a incumplir su deuda. Pero, ¿significa esto que en el futuro podremos ignorar el estado de verificación de un solicitante cuando presente una solicitud de préstamo a través de LC?

### Ejercicio 11:

¿Cuál de las siguientes afirmaciones es cierta respecto a lo que deberíamos hacer con `verification_status`, basándonos en los resultados que tenemos hasta ahora?

A. Dado que el valor $p$ de "Verification_status" es grande, podemos eliminarlo de nuestro modelo de clasificación y, en el futuro, no necesitaremos recoger esta parte de la información de los solicitantes.

B. Aunque el valor $p$ de `estado de verificación` es grande, el coeficiente de regresión estimado para él es distinto de cero. Deberíamos mantener la variable en el modelo y seguir recogiendo la información en el futuro.

C. Deberíamos llevar a cabo una evaluación adicional para comparar las precisiones de predicción del modelo con y sin "estado_de_verificación" antes de tomar cualquier otra decisión.

D. Ninguna de las anteriores.

Vamos a evaluar la precisión de la predicción utilizando el ROC. Los resultados se muestran a continuación:

In [ ]:
predict_withvs = logit_full1_res.predict(df_log2)
logit_full_novs = sm.Logit(df.loan_status.cat.codes, df_log2.loc[:, df_log2.columns != 'verification_status'])
predict_novs = logit_full_novs.fit(disp=0).predict(df_log2.loc[:, df_log2.columns != 'verification_status'])

roc_vs = roc_curve(df.loan_status.cat.codes, predict_withvs)
roc_novs = roc_curve(df.loan_status.cat.codes, predict_novs)
auc_vs = auc( roc_vs[0], roc_vs[1] )
auc_novs = auc( roc_novs[0], roc_novs[1])

plt.figure()
line_width = 2
plt.plot(roc_vs[0], roc_vs[1], color='darkorange',
         lw=line_width, label=f'With verification_status (AUC = {auc_vs:0.2f})')
plt.plot(roc_novs[0], roc_novs[1], color='darkgreen',
         lw=line_width, label=f'Without verification_status (AUC = {auc_novs:0.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=line_width, linestyle='--', label='Random guess')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Classification of Defaulters')
plt.legend(loc='lower right')
plt.show()

### Ejercicio 12:

Basándonos en todos los resultados anteriores, ¿qué podemos concluir sobre el proceso de verificación?

**Respuesta**

Encontramos que el AUC para el modelo con "estado de verificación" incluido es el mismo que el modelo sin "estado de verificación" incluido (0,69). Dado que siempre queremos que nuestro modelo sea lo más sencillo posible, esto significa que no debemos incluir `verification_status` como variable a la hora de predecir si es probable que un usuario pague su deuda.

**Connclusiones**

En este caso, investigamos si el estado de verificación de la fuente de ingresos de los solicitantes es importante a la hora de predecir si un solicitante pagará su deuda. Construimos un modelo logístico para examinar primero la relación marginal entre el estado de verificación y la probabilidad de pago. Descubrimos una tendencia contraria a la intuición: una información menos fiable sobre la fuente de ingresos estaba relacionada con una mayor probabilidad de saldar la deuda. Sin embargo, tras ajustar un modelo logístico más amplio con todas las variables independientes disponibles, descubrimos que esta tendencia se introducía a través de los efectos de confusión del tipo de interés, los ingresos anuales y el plazo del préstamo.

Después de tener en cuenta estas variables, el estado de verificación ya no estaba significativamente asociado con la probabilidad de pago. Basándonos en los valores $p$ y en el modelo ROC, concluimos que el proceso de verificación es irrelevante y que podríamos eliminarlo de los elementos requeridos y simplificar el proceso de préstamo.

En este caso, hemos introducido el concepto de **regresión logística** y cómo puede utilizarse como **clasificador**. Lo hicimos mediante:

1. Realizando EDA sobre un resultado binario utilizando tablas cruzadas
2. Realizando pruebas estadísticas para generar hipótesis
3. Codificación y comprobación de estas hipótesis con modelos más amplios mediante regresiones logísticas
4. Evaluar el rendimiento del clasificador mediante el AUC

La regresión logística amplía de forma natural el concepto de regresión lineal y es el modelo de referencia para realizar tareas de clasificación. Los coeficientes tienen una interpretación similar a los de la regresión lineal. Las curvas ROC son entonces útiles para comparar diferentes modelos de clasificación.

La regresión logística es sólo una de las muchas herramientas utilizadas en la clasificación. Aunque es simple, funciona bastante bien y necesita relativamente pocos datos para entrenarse en comparación con otros clasificadores más modernos. Aunque sólo hemos estudiado el caso binario, existen extensiones de la regresión logística para manejar también la regresión ordinal y multiclase (puede leer sobre ellas en su tiempo libre).